This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_best_worst_split(y_true, best_run, worst_run, savepath,
                          title='Best Worst Overlay', label='target_value'):
    """
    Method to create a parity plot (predicted vs. true values) of just the best scoring and worst scoring CV splits

    Args:

        y_true: (numpy array), array of true y data

        best_run: (dict), the best scoring split_result from mastml_driver

        worst_run: (dict), the worst scoring split_result from mastml_driver

        savepath: (str), path to save plots to

        title: (str), title of the best_worst_split plot

        label: (str), label used for axis labeling

    Returns:

        None

    """
    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    maxx = max(y_true) # TODO is round the right thing here?
    minn = min(y_true)
    maxx = round(float(maxx), rounder(maxx-minn))
    minn = round(float(minn), rounder(maxx-minn))
    ax.plot([minn, maxx], [minn, maxx], 'k--', lw=2, zorder=1)

    # set tick labels
    _set_tick_labels(ax, maxx, minn)

    # do the actual plotting
    ax.scatter(best_run['y_test_true'],  best_run['y_test_pred'],  c='red',
               alpha=0.7, label='best test',  edgecolor='darkred',  zorder=2, s=100)
    ax.scatter(worst_run['y_test_true'], worst_run['y_test_pred'], c='blue',
               alpha=0.7, label='worst test', edgecolor='darkblue', zorder=3, s=60)
    ax.legend(loc='lower right', fontsize=12)

    # set axis labels
    ax.set_xlabel('True '+label, fontsize=16)
    ax.set_ylabel('Predicted '+label, fontsize=16)

    #font_dict = {'size'   : 10, 'family' : 'sans-serif'}

    # Duplicate the stats dicts with an additional label
    best_stats = OrderedDict([('Best Run', None)])
    best_stats.update(best_run['test_metrics'])
    worst_stats = OrderedDict([('worst Run', None)])
    worst_stats.update(worst_run['test_metrics'])

    plot_stats(fig, best_stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, worst_stats, x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([177.28302038, 150.37102595, 202.0924573 , ...,  47.20790294,
        23.809066  ,   0.        ])
best_run = OrderedDict([('normalizer', 'DoNothing'), ('selector', 'DoNothing'), ('model', 'KernelRidge'), ('splitter', 'RepeatedKFold'), ('split_num', 9), ('y_train_true', array([150.37102595, 202.0924573 , 325.49788774, ...,  19.41019935,
        47.20790294,   0.        ])), ('y_train_pred', array([ 75.185513  , 101.04624221, 162.74896399, ...,   9.70509967,
        23.60395147,   0.        ])), ('y_test_true', array([1.77283020e+02, 2.39214774e+02, 2.14185752e+02, 1.46594936e+02,
       1.40876460e+02, 9.96717956e+01, 9.45736257e+01, 9.62792101e+01,
       2.02290968e+02, 7.75902143e+01, 1.02665208e+02, 1.72420203e+02,
       1.22487016e+02, 1.78669886e+02, 1.87709915e+02, 1.79995444e+02,
       1.53899276e+02, 1.25776102e+02, 3.69780134e+02, 3.02985863e+02,
       3.19706165e+02, 2.29722452e+02, 2.27914069e+02, 1.72040539e+02,
       1.19298374e+02, 1.24022947e+02, 7.70450205e+01, 7.45855168e+01,
       1.74631224e+02, 1.31350152e+02, 4.50731986e+01, 1.09473901e+02,
       0.00000000e+00, 2.56639133e+01, 3.90426011e+02, 2.36177315e+02,
       8.39207969e+01, 1.56229730e+02, 7.39283454e+01, 6.86792014e+01,
       1.13135302e+02, 7.34978297e+01, 1.63297865e+02, 3.20094037e+01,
       0.00000000e+00, 6.94361418e+01, 9.24561079e+01, 1.56627930e+02,
       6.36890317e+01, 5.90096117e+00, 1.56215250e+01, 7.38023139e+01,
       3.57160729e+02, 2.91632073e+01, 1.49611167e+02, 3.07830539e+02,
       2.26412639e+01, 8.03088872e+01, 5.18644462e+01, 1.39599684e+02,
       1.51797441e+02, 2.89712978e+02, 1.18302658e+02, 2.94893175e+02,
       3.39191772e+01, 3.93315709e+01, 2.99490808e+01, 2.58389489e+01,
       0.00000000e+00, 0.00000000e+00, 4.51430137e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.70304392e+01, 0.00000000e+00,
       0.00000000e+00, 1.39370004e+02, 3.20782685e+01, 3.35683107e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.49408401e+01,
       7.74558848e+00, 6.05219124e+01, 3.21439514e+01, 1.61615856e+02,
       1.31304680e+02, 1.08979858e+02, 8.64067635e+01, 0.00000000e+00,
       0.00000000e+00, 1.28935952e+02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.44184734e+01, 6.21108106e+01, 8.83802997e+01, 2.41335092e+01,
       6.53557069e+01, 4.48563810e+01, 0.00000000e+00, 1.33902387e+01,
       6.12137026e+01, 4.85711869e+01, 4.91573257e+01, 8.68896540e+01,
       3.61795100e+01, 1.04542615e+02, 1.05693165e+02, 8.65356568e+01,
       8.04591080e+01, 8.51942082e+01, 1.00949225e+02, 1.26886312e+02,
       1.85404211e+02, 0.00000000e+00, 2.02289181e-01, 1.38892855e+02,
       1.37126059e+02, 1.26881899e+02, 1.70388617e+02, 8.98167378e+01,
       9.36717096e+01, 1.50078284e+01, 1.93409720e+01, 0.00000000e+00,
       1.24480773e+01, 4.70862505e+01, 1.27378189e+02, 1.14554644e+02,
       1.66187429e+02, 1.06702335e+02, 2.76340269e+02, 0.00000000e+00,
       4.84645306e+01, 2.94415703e+02, 1.65778433e+02, 1.39763133e+02,
       3.15202492e+01, 1.40538550e+02, 2.29064683e+02, 1.13901487e+02,
       4.80411705e+01, 2.53031123e+01, 3.81465155e+01, 6.49749186e+01,
       4.22355880e+01, 2.62303388e+01, 9.53324888e+01, 9.10665100e+00,
       0.00000000e+00, 1.05110827e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.84401941e+01, 5.54728596e+01, 5.13197137e+01,
       5.25266742e+01, 4.81837797e+01, 3.08078053e+01, 2.02393592e+02,
       5.65407126e+01, 3.81230146e+01, 5.46661575e+01, 5.87368740e+01,
       1.29976759e+02, 4.16645268e+01, 2.82496436e+01, 3.40111139e+01,
       0.00000000e+00, 6.50315409e+01, 1.89559087e+01, 3.92079232e+01,
       1.52429429e+02, 9.46538849e+01, 8.30269047e+01, 0.00000000e+00,
       4.00673869e+01, 2.50852147e+01, 3.11645320e+01, 2.75959946e+01,
       0.00000000e+00, 9.54966739e+01, 4.94252903e+01, 1.05442368e+02,
       0.00000000e+00, 0.00000000e+00, 6.89012756e+00, 0.00000000e+00,
       0.00000000e+00, 7.05592531e+00, 5.40607889e+01, 6.90074178e+01,
       0.00000000e+00, 0.00000000e+00, 1.29601532e+02, 1.55663046e+02,
       1.36431444e+02, 1.89093084e+02, 1.79654132e+02, 2.07904807e+02,
       2.34660651e+02, 3.36224697e+02, 9.15340044e+01, 1.16180977e+02,
       1.78939785e+02, 1.14272223e+02, 1.13869162e+02, 2.43428371e+02,
       1.14407716e+02, 1.86026530e+02, 1.30347188e+02, 1.26700822e+02,
       2.44148911e+02, 1.55694637e+02, 2.59886084e+02, 2.20161511e+02,
       1.96409134e+02, 1.44708713e+02, 1.91836674e+02, 1.41535230e+02,
       2.10335728e+02, 1.03165153e+02, 1.35390845e+02, 2.51196620e+02,
       2.37062344e+02, 2.73613179e+02, 3.62461054e+02, 1.77346156e+02,
       2.26507616e+02, 3.23822637e+02, 2.04046571e+02, 2.23816699e+02,
       3.40437655e+02, 2.55478983e+02, 3.18399908e+02, 5.98511381e+01,
       1.43517205e+02, 8.23163755e+01, 1.97124130e+02, 9.69915119e+01,
       2.10107835e+02, 1.61293351e+02, 9.69267962e+01, 9.83944608e+01,
       1.93969856e+02, 1.35408933e+02, 1.31977595e+02, 1.83449190e+02,
       1.02794584e+02, 1.60233988e+02, 3.64707101e+02, 1.86717789e+02,
       1.44889042e+02, 3.17205543e+02, 3.17356871e+02, 3.28541723e+02,
       3.23634151e+02, 3.46087908e+02, 1.20403530e+02, 1.41453455e+02,
       7.10448978e+01, 8.95779086e+01, 1.24842572e+02, 8.85969515e+01,
       1.38575932e+02, 1.36443088e+02, 1.47525298e+02, 6.40241718e+01,
       5.55045542e+01, 1.38091226e+02, 7.74665625e+01, 1.33883367e+02,
       1.32392433e+02, 1.26818170e+02, 5.68526124e+01, 1.84907725e+02,
       1.25571144e+02, 2.72871580e+02, 2.10329787e+02, 2.83042429e+02,
       2.27717761e+02, 2.47909595e+02, 2.42965465e+02, 2.00760493e+02,
       2.04542101e+02, 3.18785354e+02, 2.71906598e+02, 2.09325439e+02,
       1.35426489e+02, 8.10050964e+01, 5.74165434e+01, 1.04682468e+02,
       9.45208382e+01, 9.43913417e+01, 1.78159873e+02, 9.64735725e+01,
       2.72145096e+01, 6.07829098e+01, 2.42317220e+01, 4.96708469e+01,
       7.34595746e+01, 7.19683460e+01, 1.82785945e+01, 4.28892788e+01,
       3.27734114e+01, 1.35840083e+02, 1.02112315e+02, 1.73385595e+02,
       1.84215977e+02, 1.97176332e+02, 1.84779292e+02, 1.70292754e+02,
       1.63409286e+02, 1.63793911e+02, 1.11384136e+02, 1.54672899e+02,
       1.15407866e+02, 1.81387575e+02, 0.00000000e+00, 2.98870605e+01,
       7.73860277e+01, 9.14620807e+01, 8.35686242e+01, 9.06089707e+01,
       2.71319339e+02, 1.09488956e+02, 9.60360384e+01, 2.58828051e+02,
       1.17249168e+02, 1.37791901e+02, 1.85661694e+02, 1.12287386e+02,
       1.40364781e+02, 1.37369485e+02, 1.21692419e+02, 3.35321293e+02,
       9.37481720e+01, 2.63543371e+02, 4.21335070e+01, 3.63667394e+01,
       2.90221571e+02, 2.71330052e+01, 4.45796202e+00, 1.91667256e+01,
       9.53050436e+01, 3.73225941e+01, 1.12949106e+02, 5.04471564e+01,
       1.34244899e+02, 1.43075475e+02, 1.40737465e+02, 1.40488835e+02,
       1.04314376e+01, 1.43739984e+02, 1.61966268e+02, 1.78944559e+02,
       1.82449930e+02, 3.87496682e+01, 3.50703870e+01, 6.20947510e+01,
       6.92239663e+00, 0.00000000e+00, 0.00000000e+00, 6.23247673e+01,
       4.92192870e+01, 5.92192098e+01, 7.32347672e+01, 1.60334633e+02,
       5.19157130e+01, 2.84812922e+01, 2.83147240e+01, 2.55044144e+00,
       3.23684503e+01, 4.56348093e+01, 0.00000000e+00, 3.86517633e+01,
       2.99157066e+01, 7.07659236e+01, 2.67038101e+01, 3.37558922e+01,
       3.74680233e+01, 8.17268550e+01, 7.65032664e+01, 7.90798034e+01,
       7.72377141e+01, 6.23324679e+01, 5.22740325e+01, 1.21693503e+02,
       3.81925114e+01, 5.21415070e+01, 0.00000000e+00, 4.44093613e+01,
       4.39454900e+01, 3.91274706e+01, 1.90870878e+02, 1.66347669e+02,
       8.65192126e+01, 8.25826337e+01, 1.77019953e+01, 1.05624926e+02,
       3.62234796e+01, 1.42323884e+02, 3.48683785e+01, 4.63580983e+01,
       3.07458097e+01, 5.78409146e+01, 4.67667976e+01, 3.53717988e+01,
       5.24254612e+01, 4.66982172e+01, 2.28817925e+01, 8.97135102e+01,
       7.80670028e+01, 2.19120080e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.88957798e+01, 0.00000000e+00,
       2.38090660e+01])), ('y_test_pred', array([ 2.46824268e-005,  1.48720127e-005,  1.25837471e-270,
        8.78222560e-135,  3.28018776e-133,  2.06042835e-231,
        7.94081189e-028,  3.97993955e-192,  4.30335593e-231,
        1.98940071e-027,  3.39871916e-231,  0.00000000e+000,
        0.00000000e+000,  2.74497073e-130,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        3.24338957e-010,  5.14599817e-236,  4.78919061e-236,
        3.07381952e-304,  0.00000000e+000,  0.00000000e+000,
        2.10224626e-032,  8.42027393e-148,  1.48633439e-032,
        0.00000000e+000,  1.05517298e-252,  1.59223042e-036,
        2.95120309e-242,  3.10265696e-029,  6.58045386e-003,
        4.27152655e-038,  9.20767117e-137,  9.92729010e-111,
        3.71369616e-025,  1.08192522e-003,  3.56856916e-022,
        2.13492880e-021,  1.84375346e-010,  1.27840792e-013,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        1.40573846e-038,  0.00000000e+000,  0.00000000e+000,
        1.89981767e-098,  1.35588136e-075,  0.00000000e+000,
        8.21371737e-027,  0.00000000e+000,  1.57577633e-004,
        1.04022784e-020,  1.31274761e-036,  2.22705163e-005,
        0.00000000e+000,  3.50877210e-011,  5.30490469e-024,
        8.64306811e+000,  0.00000000e+000,  1.37469631e-005,
        2.18213044e-009,  7.07612205e-001,  9.92572461e-001,
        5.38578288e-002,  5.44776787e-249,  0.00000000e+000,
       -7.18870662e-239,  4.73036284e-238,  5.89732765e-239,
        5.31358639e-006, -9.22853000e-009, -2.76076032e-005,
        5.48357800e-004,  4.94245387e-251,  5.63977402e-095,
        9.57739162e-011,  1.85675892e-012,  2.83541773e-007,
        3.31623371e-242,  2.65759088e-242,  8.29806067e-008,
        2.58578678e-007,  2.03445047e-010,  2.87365010e-072,
        1.31411899e-011,  2.29517165e+000,  1.24751592e-020,
        9.04714408e-009,  0.00000000e+000,  0.00000000e+000,
        5.09400716e-248,  6.87664887e-014,  5.26793332e-238,
       -4.69803534e-258,  5.58162334e-240,  3.64212853e-255,
        2.62492659e-244,  3.26018949e-104,  3.83666377e-012,
        6.91097764e-009,  8.19500489e-013,  1.99731957e-003,
        1.71695051e-010,  0.00000000e+000,  1.49563477e-006,
        8.20419406e-003,  2.42091093e+000,  2.24321298e-007,
        3.09134622e-034,  9.46572441e-015,  7.54515501e-027,
        0.00000000e+000,  1.35750262e-162,  9.73511720e-287,
        0.00000000e+000,  1.03312794e-022,  3.31692117e-011,
        4.66390953e-005,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  3.05833775e-025,  1.00098061e-022,
        0.00000000e+000,  4.49083689e+001,  8.85411804e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  9.63009969e-014,  7.45936681e-023,
        2.02896282e-110,  2.63545356e-048,  8.70887330e-023,
        4.33126605e-005,  0.00000000e+000,  3.54159428e-008,
        1.47812937e-028,  9.31103473e-003,  3.28189038e-013,
        3.10207527e-098,  3.30773736e-047,  1.23908030e-079,
        4.87473896e-088,  1.74271768e-006,  6.30098276e+000,
        8.86925717e-237,  2.23118754e-234,  8.77518228e-001,
        5.63770956e-002,  1.77540981e-008,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000, -5.73417739e-011,
        0.00000000e+000, -3.33018493e-017,  3.24803541e-252,
        1.23272672e-101,  1.75755223e-006,  6.79515670e-008,
        7.91180825e-004,  3.05088379e-003,  1.39148106e-076,
        1.29362111e-006,  7.17100678e-006,  1.53778008e-069,
        1.69911835e-030,  4.29464043e-051,  7.18690202e-004,
        0.00000000e+000,  2.50949562e-239,  0.00000000e+000,
        4.70761658e-033,  6.87568961e-021,  1.23612033e-253,
        1.99852667e-076,  3.48342036e-239,  3.86094403e-239,
        1.14522347e-004,  6.40976246e-008,  6.14916987e+000,
        2.54432688e-017,  4.10574879e-017,  0.00000000e+000,
        2.00379770e-006,  1.10932723e-063,  1.74874865e-004,
        5.63306841e-013,  1.68858898e-198,  2.40196087e-198,
        0.00000000e+000,  2.80827713e-199,  2.39317091e-198,
        1.60942093e-003,  7.03817567e-032,  5.43277992e-204,
        0.00000000e+000,  2.73855791e-001,  3.52339917e-008,
        0.00000000e+000,  4.68835581e+000,  2.55638927e-001,
        6.39647127e-020,  4.63950126e-008,  8.15440091e+000,
        2.04901185e-001,  2.85846476e-001,  2.24448289e-005,
        8.22187280e+000,  3.16552976e+000,  6.45845271e-010,
        2.77804681e+000,  4.75718022e+000,  3.19311664e+000,
        2.50888735e-001,  2.87503449e-001,  3.62198440e-001,
        6.04152957e-001,  5.52259807e-002,  1.75712630e-020,
        1.23958296e-020,  2.08197484e-008,  1.46857067e-008,
        5.14129506e+000,  2.63042658e+000,  2.70136542e+000,
        5.14886102e+000,  5.01701368e-001,  5.67465023e-001,
        7.79460020e-001,  4.78377799e-002,  5.42865052e-002,
        8.93722558e+000,  6.37583699e+000,  6.86364339e+000,
        1.08857147e-010,  5.20155306e+000,  6.24440883e-004,
        3.13322578e+000,  4.21558239e+000,  1.73688711e-020,
        3.47558148e-001,  1.36116235e-001,  2.05946350e-008,
        2.86524080e-001,  1.92623836e-001,  1.99023767e-001,
        3.69570465e-001,  7.40034063e-011,  1.88507265e-007,
        1.90129753e+001,  2.58588668e+000,  8.20012226e+001,
        7.53021801e-001,  4.59141850e-002,  6.96701067e-002,
        8.60737285e+000,  9.58185912e+000,  1.00964981e+001,
        6.79472273e+000,  7.77850482e+000,  3.00230730e+000,
        3.46535214e+000,  1.91385873e+000,  2.53437805e+000,
        2.58979566e-001,  3.26719230e-001,  3.16565244e-008,
        3.28993927e-002,  3.69114390e-002,  1.24167910e-001,
        3.04110518e-001,  3.28182995e-008,  3.58462331e-008,
        2.87975286e-005,  1.30272279e+001,  8.61928170e-002,
        6.20996534e+000,  3.67924881e+000,  1.66715658e-004,
        2.49415065e-023,  6.33969766e-001,  1.80801117e-011,
        5.56465695e-002,  6.08385375e-002,  6.57950206e+000,
        5.29482942e+000,  6.87842009e-033,  7.96564105e-011,
        8.33503849e-002,  5.42324939e-071,  3.10550188e-008,
        1.97822969e-006,  2.42981258e-017,  4.62685175e-104,
        7.03272873e-004,  7.94668728e-004,  1.47742493e-003,
        7.42015568e-004,  6.35427799e-070,  3.50897491e-006,
        5.21312642e-017,  3.07859231e-247,  2.58357082e-109,
        5.89661445e-004,  2.11616673e-004,  4.52619153e-004,
        1.71213881e-004,  4.84610805e-025,  2.46687204e-103,
        5.30636196e-015,  8.65750779e-012,  6.83302438e-116,
        1.04203889e-038,  5.52448150e-015,  1.39726440e-003,
        5.14035141e+000,  9.51452316e-004,  1.33365812e-003,
        1.06823103e-003,  4.31157829e-010,  4.27179136e-114,
        2.97856703e-028,  0.00000000e+000,  2.63345382e+000,
        2.78834644e-005,  8.34729323e-002,  0.00000000e+000,
        1.27354473e-003,  1.04381729e+001,  2.30427130e-003,
        2.84646084e+000,  8.85514140e-004,  4.47931328e+000,
        1.14885217e-004,  3.42363919e+000,  1.36532681e+001,
        1.12069191e-001,  2.47269338e-004,  4.84688808e-004,
        0.00000000e+000,  9.34471177e-001,  8.84910428e-024,
        1.87548975e-217,  6.67679702e-018,  2.85931610e-019,
        2.79641264e-020,  1.01165080e-184,  1.65322448e-015,
        4.41487672e-005,  1.42309455e-002,  5.93610104e-019,
        1.94592911e-124,  0.00000000e+000,  8.46374012e-062,
        0.00000000e+000,  5.86555998e-006,  9.72828140e-062,
        4.47663279e-105,  0.00000000e+000,  3.15431582e-092,
        0.00000000e+000,  3.51707333e-008,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  7.77815139e-078,
        7.44364292e-028,  7.64574663e-031,  2.19661735e-001,
        2.95822179e+000,  1.30655194e-001,  2.82124452e-028,
        2.88924640e-031,  2.45638971e-018,  4.26469187e-003,
        6.20529438e-078,  0.00000000e+000,  4.53656851e-011,
        9.95838225e-018,  3.05434950e-063,  1.11302353e-005,
        3.33169539e-002,  1.67364651e+000,  9.99141628e-028,
        1.12669156e-030,  1.09744458e-010,  2.18478693e-017,
        8.94566928e-006,  5.73128771e-006,  4.31864788e-051,
        3.24250875e-078,  6.00981790e-028,  0.00000000e+000,
        1.11722543e-017,  7.38263966e-003,  3.90465660e+000,
        3.89135007e-080,  1.02627747e-077,  1.18312935e-030,
        1.99477234e-017,  1.85843993e-008,  3.18997713e+000,
        3.21960342e-004,  7.04049630e-078,  5.43540001e-028,
        5.08243692e-031,  4.50057844e-011,  8.81497450e-071,
        1.27623698e-069,  3.69489015e-019,  7.28800816e-031,
        6.43694779e-011,  2.07685653e-063,  6.11422121e-009,
        3.23202800e-006,  1.16496389e-030,  0.00000000e+000,
        0.00000000e+000, -1.85218466e-177,  0.00000000e+000,
        0.00000000e+000,  2.51722038e-006,  3.01387771e-027])), ('train_metrics', OrderedDict([('R2', 0.9973160562585882), ('root_mean_squared_error', 71.63585850788199), ('mean_absolute_error', 52.74921212980539), ('rmse_over_stdev', 0.7369986413540602)])), ('test_metrics', OrderedDict([('R2', 0.018889512167304834), ('root_mean_squared_error', 136.26261102873644), ('mean_absolute_error', 104.62829468730243), ('rmse_over_stdev', 1.4018867266661694)])), ('train_indices', array([   1,    2,    3, ..., 2140, 2142, 2144])), ('test_indices', array([   0,   10,   12,   37,   39,   46,   48,   51,   58,   65,   72,
         92,   98,  109,  110,  111,  112,  115,  118,  121,  123,  124,
        125,  126,  132,  133,  144,  148,  154,  160,  164,  167,  170,
        176,  187,  190,  193,  212,  218,  221,  234,  250,  253,  254,
        256,  262,  264,  272,  273,  278,  292,  297,  300,  315,  317,
        323,  327,  333,  339,  346,  356,  359,  360,  368,  379,  388,
        393,  397,  406,  408,  412,  442,  445,  453,  454,  458,  459,
        463,  465,  469,  480,  483,  484,  485,  489,  498,  507,  530,
        532,  538,  539,  544,  545,  549,  558,  561,  566,  570,  571,
        573,  583,  594,  601,  611,  613,  624,  627,  632,  636,  639,
        640,  647,  648,  652,  658,  665,  667,  674,  683,  686,  689,
        692,  693,  694,  700,  703,  704,  706,  710,  712,  713,  715,
        716,  722,  725,  727,  728,  734,  740,  744,  747,  753,  757,
        759,  764,  771,  772,  782,  785,  792,  795,  796,  800,  806,
        810,  817,  819,  821,  828,  834,  835,  840,  844,  847,  851,
        855,  860,  887,  888,  892,  900,  901,  905,  918,  920,  921,
        931,  933,  934,  935,  950,  954,  955,  963,  973,  976,  981,
        988, 1015, 1020, 1021, 1023, 1026, 1039, 1040, 1041, 1043, 1044,
       1052, 1053, 1054, 1069, 1072, 1079, 1081, 1092, 1095, 1096, 1097,
       1113, 1117, 1120, 1122, 1125, 1127, 1133, 1138, 1142, 1144, 1150,
       1151, 1163, 1164, 1173, 1175, 1178, 1180, 1183, 1197, 1198, 1204,
       1206, 1217, 1219, 1220, 1222, 1225, 1226, 1233, 1234, 1236, 1239,
       1241, 1249, 1250, 1261, 1262, 1264, 1266, 1277, 1286, 1292, 1294,
       1301, 1306, 1307, 1309, 1320, 1328, 1335, 1337, 1341, 1343, 1347,
       1351, 1353, 1359, 1360, 1361, 1363, 1368, 1372, 1378, 1391, 1393,
       1397, 1398, 1404, 1407, 1408, 1409, 1413, 1416, 1427, 1431, 1433,
       1437, 1438, 1443, 1444, 1448, 1451, 1461, 1462, 1463, 1470, 1476,
       1486, 1492, 1497, 1507, 1510, 1515, 1526, 1539, 1547, 1552, 1555,
       1556, 1559, 1567, 1569, 1583, 1597, 1599, 1603, 1607, 1608, 1615,
       1618, 1624, 1625, 1629, 1630, 1632, 1633, 1650, 1652, 1653, 1655,
       1659, 1660, 1662, 1669, 1672, 1674, 1680, 1682, 1692, 1702, 1717,
       1721, 1723, 1724, 1732, 1737, 1742, 1751, 1752, 1757, 1765, 1767,
       1768, 1770, 1780, 1783, 1788, 1796, 1797, 1800, 1807, 1811, 1812,
       1820, 1821, 1822, 1824, 1834, 1840, 1844, 1845, 1858, 1860, 1865,
       1872, 1875, 1884, 1892, 1896, 1900, 1912, 1918, 1922, 1927, 1928,
       1931, 1939, 1940, 1941, 1949, 1952, 1955, 1956, 1963, 1967, 1977,
       1980, 1983, 1985, 1991, 1997, 2000, 2009, 2011, 2017, 2020, 2026,
       2031, 2032, 2043, 2044, 2049, 2051, 2058, 2070, 2082, 2084, 2086,
       2090, 2091, 2094, 2108, 2113, 2118, 2124, 2130, 2133, 2141, 2143])), ('train_groups', None), ('test_groups', None), ('prediction_metrics', None)])
worst_run = OrderedDict([('normalizer', 'DoNothing'), ('selector', 'DoNothing'), ('model', 'KernelRidge'), ('splitter', 'RepeatedKFold'), ('split_num', 0), ('y_train_true', array([177.28302038, 202.0924573 , 325.49788774, ...,  47.20790294,
        23.809066  ,   0.        ])), ('y_train_pred', array([ 88.64152247, 101.04624221, 162.74894387, ...,  23.60395147,
        11.904533  ,   0.        ])), ('y_test_true', array([1.50371026e+02, 2.22927321e+02, 2.29790128e+02, 2.41537703e+02,
       4.07147186e+02, 3.46912901e+02, 2.44558934e+02, 7.98843255e+01,
       9.96717956e+01, 1.20156284e+02, 1.06566685e+02, 1.86835775e+02,
       7.75902143e+01, 1.78590946e+02, 1.02665208e+02, 1.27306711e+02,
       1.47491884e+02, 1.01679243e+02, 1.70112526e+02, 1.71679718e+02,
       1.44367703e+02, 1.90002071e+02, 1.90460889e+02, 1.25776102e+02,
       1.72040539e+02, 1.85728826e+02, 1.37488659e+02, 7.23673571e+01,
       1.74631224e+02, 3.79332462e+01, 9.24493164e+01, 0.00000000e+00,
       9.78662706e+01, 9.51442246e+01, 2.36177315e+02, 1.08666855e+02,
       7.09734216e+01, 2.91384616e+02, 6.60766101e+01, 2.36664682e+02,
       4.08488586e+02, 5.15805330e+02, 1.22389663e+02, 8.81149314e+01,
       1.57715585e+02, 2.37521275e+02, 1.28364345e+02, 2.38334229e+02,
       3.47383100e+02, 4.16012879e+01, 9.79676617e+00, 7.48766194e+00,
       0.00000000e+00, 1.56215250e+01, 6.31512971e+01, 0.00000000e+00,
       5.64799569e+01, 1.49611167e+02, 0.00000000e+00, 0.00000000e+00,
       3.11290486e+01, 7.93342932e+01, 5.04901919e+01, 0.00000000e+00,
       2.12302521e+02, 1.98106664e+02, 1.18302658e+02, 3.77502009e+02,
       1.19774983e+02, 5.00254523e+01, 0.00000000e+00, 3.31384554e+01,
       4.09146349e+01, 3.93315709e+01, 4.89115430e+01, 0.00000000e+00,
       0.00000000e+00, 4.51430137e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.69769380e+02, 1.08224668e+02, 3.34361123e+01,
       4.06187327e+01, 6.70851015e+01, 4.33637207e+01, 3.81461657e+01,
       3.21439514e+01, 3.92412752e+01, 2.10808929e+02, 2.32209209e+02,
       1.57374363e+02, 0.00000000e+00, 1.55792869e+02, 9.57636772e+01,
       0.00000000e+00, 0.00000000e+00, 6.92693877e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.06802766e+01,
       5.53782777e+01, 4.07973166e+01, 5.08263675e+01, 1.41470544e+02,
       2.39414173e+01, 7.92325765e+01, 6.53557069e+01, 3.98312067e+01,
       2.18803383e+01, 4.48563810e+01, 1.33902387e+01, 3.61795100e+01,
       1.44194572e+02, 1.07975067e+02, 1.12573845e+02, 9.85807552e+01,
       0.00000000e+00, 2.83906453e+02, 2.02289181e-01, 1.51553671e+02,
       5.24822446e+01, 8.98167378e+01, 8.77843631e+01, 9.36717096e+01,
       0.00000000e+00, 8.69178900e+00, 4.36717688e+00, 1.27378189e+02,
       1.71956703e+02, 2.58665310e+02, 0.00000000e+00, 4.21689745e+01,
       3.15202492e+01, 1.26777344e+02, 2.29064683e+02, 1.99684569e+02,
       0.00000000e+00, 1.38924377e+01, 5.14231024e+01, 0.00000000e+00,
       0.00000000e+00, 7.81817223e+01, 4.55261914e+01, 4.79204659e+01,
       0.00000000e+00, 5.09486685e+01, 6.98113911e+01, 1.55465078e+02,
       1.32773863e+02, 2.04099993e+02, 6.77439450e+01, 7.83502295e+01,
       7.67577252e+01, 6.94770317e+01, 4.42032278e+01, 3.74738040e+01,
       5.46661575e+01, 5.67330262e+01, 0.00000000e+00, 0.00000000e+00,
       3.95429854e+01, 2.82496436e+01, 0.00000000e+00, 3.42510607e+01,
       1.62892176e+02, 1.50562981e+02, 0.00000000e+00, 1.41471166e+02,
       1.95442993e+02, 2.85835223e+01, 1.15904985e+01, 5.50100622e+01,
       7.11848257e+01, 1.03266329e+02, 6.89092871e+01, 0.00000000e+00,
       3.69821609e+00, 5.31766390e-01, 1.12780820e+01, 0.00000000e+00,
       0.00000000e+00, 4.45467226e+01, 7.37535702e+01, 0.00000000e+00,
       2.44348237e+02, 1.36431444e+02, 1.85197855e+02, 2.02269808e+02,
       2.65270592e+02, 1.32871922e+02, 3.78453371e+02, 8.83272115e+01,
       9.03169483e+01, 9.77903794e+01, 2.41664953e+02, 1.81295285e+02,
       2.49724729e+02, 2.30118336e+02, 1.14407716e+02, 1.31761462e+02,
       1.89952208e+02, 1.24477908e+02, 1.26700822e+02, 1.49948333e+02,
       1.55694637e+02, 1.50672468e+02, 1.63799175e+02, 2.18442616e+02,
       9.81125625e+01, 1.52280199e+02, 1.91836674e+02, 2.12302508e+02,
       1.78586168e+02, 1.01768108e+02, 1.56136057e+02, 1.35390845e+02,
       1.69573331e+02, 2.73613179e+02, 2.37992715e+02, 3.09856621e+02,
       2.04046571e+02, 2.55478983e+02, 3.49453381e+02, 3.18399908e+02,
       7.64960376e+01, 2.05564030e+02, 7.04378967e+01, 8.23163755e+01,
       9.77091114e+01, 8.35495499e+01, 2.08659157e+02, 1.77317832e+02,
       2.80854292e+02, 2.06610134e+02, 1.54641811e+02, 1.00117427e+02,
       1.40265182e+02, 9.48376021e+01, 9.54547321e+01, 1.83449190e+02,
       2.13333228e+02, 1.08425980e+02, 1.59401405e+02, 1.85807040e+02,
       2.40576558e+02, 2.59216571e+02, 3.15514875e+02, 1.44889042e+02,
       2.38988178e+02, 2.52611496e+02, 2.18137285e+02, 3.05414508e+02,
       7.10448978e+01, 7.89063338e+01, 6.56442278e+01, 1.70005981e+02,
       1.38575932e+02, 1.79659313e+02, 1.18314107e+02, 1.23035360e+02,
       1.36443088e+02, 5.95241562e+01, 1.29155595e+02, 5.68526124e+01,
       1.65456460e+02, 3.12115625e+02, 2.42965465e+02, 1.91404536e+02,
       2.00760493e+02, 9.56831956e+02, 1.28639131e+02, 1.23367070e+02,
       1.38844996e+02, 5.74165434e+01, 1.04682468e+02, 1.10065846e+02,
       1.43722366e+02, 1.09913097e+02, 1.94691060e+02, 1.15142225e+02,
       1.51089123e+02, 9.45208382e+01, 1.26955312e+02, 1.78159873e+02,
       1.56518089e+02, 9.04495542e+01, 6.89800768e+01, 2.66189733e+01,
       5.18182329e+01, 3.70668895e+01, 1.24273117e+02, 2.42317220e+01,
       3.26473248e+01, 7.34595746e+01, 7.19683460e+01, 4.18854297e+01,
       4.30159281e+01, 7.21745594e+01, 4.85930390e+01, 9.42635786e+01,
       4.29043523e+01, 1.73730028e+02, 1.76433607e+02, 2.10616630e+02,
       1.60456727e+02, 1.67324805e+02, 2.11267960e+02, 1.87207178e+02,
       2.36861914e+02, 1.33312148e+02, 1.54672899e+02, 2.00391998e+02,
       9.14620807e+01, 1.00815664e+02, 2.71319339e+02, 1.37479653e+02,
       1.10285836e+02, 1.35954460e+02, 1.83271284e+02, 1.17247059e+02,
       1.37791901e+02, 1.56027039e+02, 1.35955483e+02, 2.16122195e+02,
       1.36808521e+02, 1.35369317e+02, 1.21692419e+02, 9.37481720e+01,
       7.98648319e+01, 1.34354983e+02, 3.33198729e+01, 2.83093128e+02,
       2.23792303e+01, 1.09582964e+02, 9.88464407e+01, 1.40488835e+02,
       1.11369551e+02, 1.92009283e+02, 1.04314376e+01, 9.43179777e+01,
       1.32019981e+02, 1.78944559e+02, 9.04350450e+01, 0.00000000e+00,
       1.55909864e+02, 9.38423029e+01, 7.18885960e+01, 9.37906658e+01,
       5.63388158e+01, 4.54043655e+01, 6.24481656e+01, 0.00000000e+00,
       1.01755733e+02, 2.83147240e+01, 2.55044144e+00, 6.11195316e+01,
       2.86637589e+01, 7.99376675e+00, 0.00000000e+00, 3.23684503e+01,
       1.20026452e+02, 3.86517633e+01, 2.77992144e+01, 1.72121127e+01,
       0.00000000e+00, 0.00000000e+00, 1.84387405e+02, 7.65032664e+01,
       6.74148765e+01, 6.23324679e+01, 3.60966810e+01, 9.16707430e+01,
       1.16752352e+02, 4.08206874e+01, 4.60103452e+01, 3.88036349e+01,
       8.61157898e+01, 8.42613186e+01, 8.65192126e+01, 5.33006835e+01,
       6.76609259e+01, 1.05624926e+02, 4.11379073e+01, 2.72516127e+01,
       4.63580983e+01, 3.15042600e+01, 3.05960388e+01, 4.59371875e+01,
       2.93690973e+01, 9.40057842e+01, 4.67667976e+01, 1.20061041e+02,
       2.84167104e+01, 4.54076116e+01, 4.53272821e+01, 5.00275181e+01,
       8.97135102e+01, 1.25245007e+02, 7.00096850e+00, 7.80670028e+01,
       7.76829500e+01, 1.34709235e+01, 5.97432500e+00, 7.83039861e+01,
       2.19120080e+01, 7.17549475e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 8.66887169e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])), ('y_test_pred', array([ 2.37162407e-005,  0.00000000e+000,  1.57670412e-122,
        0.00000000e+000,  3.21633711e-005,  5.24841590e-236,
        0.00000000e+000,  4.54344107e-028,  2.06042835e-231,
        5.60757180e-192,  0.00000000e+000,  8.99141021e-133,
        1.98940071e-027,  3.94752770e-027,  3.39871916e-231,
        1.05056155e-132,  4.59397478e-192,  0.00000000e+000,
        7.51332792e-192,  0.00000000e+000,  1.93017472e-010,
        2.25516103e-130,  8.03909803e-236,  0.00000000e+000,
        0.00000000e+000,  5.69420679e-032,  1.07967237e-132,
        0.00000000e+000,  1.05517298e-252,  7.66926972e-258,
        6.18256310e-013,  1.53153616e-157,  3.41500373e-001,
        5.12641420e-027,  9.92729010e-111,  1.59368346e-103,
        0.00000000e+000,  1.74801913e-115,  1.61738315e-020,
        7.14212545e-004,  3.48980138e-044,  8.33153752e-125,
        1.00100267e-090,  1.49372738e-022,  1.84682607e-092,
        4.20249123e-002,  9.65766292e-006,  1.17461841e-025,
        5.39575881e-040,  9.35854752e-099,  0.00000000e+000,
        1.29774479e-106,  3.23319374e-108,  0.00000000e+000,
        9.59903241e-027,  8.25336855e-011,  4.42365260e-003,
        1.04022706e-020,  0.00000000e+000,  4.30929508e-180,
        3.50452414e-067,  3.53193397e-029,  1.46674629e-023,
        2.08741624e-054,  0.00000000e+000,  9.51277628e-045,
        1.37469631e-005,  0.00000000e+000,  1.58884929e-001,
        5.72914762e-095,  0.00000000e+000,  3.32575071e-072,
        1.43238441e-012,  5.32517745e-006,  9.95516546e-002,
        8.99453929e-239,  5.74151822e-241,  4.73036284e-238,
        0.00000000e+000,  0.00000000e+000,  6.58523341e-031,
       -1.48090289e-105, -4.94570825e-076,  4.52271122e-097,
        2.69747241e-235,  7.70760338e-239, -1.28914187e-007,
        0.00000000e+000,  5.59586356e-002,  3.90799562e-084,
        1.72505857e-033,  5.99006251e-008,  1.22193215e-012,
        9.64462759e-117,  3.19095140e-084,  2.49239196e-095,
        2.03563057e-303,  2.17502467e-239,  6.70779449e-084,
        1.35637112e-094,  6.86576022e-010,  0.00000000e+000,
        1.43118692e-071,  7.11505024e-005,  0.00000000e+000,
        0.00000000e+000,  2.23444414e+000, -2.14724096e-146,
        1.34949156e-241,  5.75700245e-240,  7.91172700e-240,
        2.09913303e-242,  2.27881472e-242, -3.94692655e-088,
        6.83274234e-096,  1.98846757e-010,  4.30310055e-012,
        6.10181038e-001,  2.41910045e-006,  4.31746609e-009,
        3.19772012e-089,  2.41692700e-004,  1.99731957e-003,
        1.44580073e-030,  2.53105552e-095,  1.71695051e-010,
        1.49563477e-006,  5.50334199e-003,  2.40953996e-161,
        0.00000000e+000,  1.04351398e-162,  0.00000000e+000,
        5.76416295e-007,  3.04575136e-005,  4.41746395e-094,
        6.01608154e-049,  3.46064758e-006,  4.49083689e+001,
        7.78285888e-010,  9.07968332e-010,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  7.45936682e-023,
        3.15327144e-213,  1.69322763e-048,  1.14122705e-056,
        5.58005897e-156,  3.10207527e-098,  6.18648971e-010,
        4.27243789e-011,  1.33494490e-023,  0.00000000e+000,
        0.00000000e+000,  1.77543543e-240, -1.13506940e-027,
        0.00000000e+000,  9.81655039e-088,  3.04281346e-011,
        1.13014587e-004,  0.00000000e+000,  4.89465128e+000,
        7.94957609e-004,  1.83111272e-087,  0.00000000e+000,
        0.00000000e+000,  1.42764085e-237,  7.37555700e-239,
        1.18108619e-007,  1.39166377e-003,  6.18462520e-006,
        2.77400168e-026,  4.56234169e-007,  1.73944097e-006,
        0.00000000e+000,  0.00000000e+000,  9.99740503e+000,
        3.46459644e-077,  8.13512929e-009,  5.67010675e-003,
        1.67141215e-087,  4.75936101e-006,  0.00000000e+000,
        1.99618584e-007,  1.55868612e-076,  4.49020499e-004,
        7.91391488e-089,  3.16055502e-002,  9.06351315e-133,
        5.17850042e-133,  1.68368267e-003,  0.00000000e+000,
        5.84639034e+000,  4.55445253e-078,  7.71465973e-003,
        2.79718708e-199,  0.00000000e+000,  9.13582395e-062,
        6.64782738e-027,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  4.05099155e+000,  6.26723852e+000,
        7.35982868e-011,  3.45645532e-001,  7.67550218e-001,
        6.10556997e-032,  4.09243806e-020,  2.11922523e-001,
        5.36558010e-001,  2.21316506e-005,  0.00000000e+000,
        0.00000000e+000,  2.77804681e+000,  3.15884301e+000,
        3.65249042e-001,  2.36213365e-001,  2.50888735e-001,
        4.68118952e-001,  3.62198440e-001,  2.62778285e-001,
        2.94240547e-001,  5.62908337e-002,  2.37057224e-001,
        2.77050197e-001,  3.77843055e-001,  2.06843597e+001,
        1.91693684e+001,  1.07130365e+001,  3.95419122e+000,
        2.70103344e+000,  4.45970404e-004,  5.67465023e-001,
        6.45481106e+000,  9.34004877e+000,  6.37583699e+000,
        5.20155306e+000,  7.50160537e+000,  7.57432041e+000,
        1.86746244e+000,  4.55897313e+000,  2.12051267e-001,
        4.03892147e-001,  4.41927862e+001,  3.21279779e-001,
        3.52821994e-001,  4.47896744e-002,  4.07735967e-008,
        3.82071506e-001,  2.73069006e-001,  1.86485140e-001,
        3.19385646e-008,  9.52423662e+000,  1.01629204e+001,
        1.90129753e+001,  5.22669044e+000,  2.69381775e+000,
        3.53507004e-047,  2.61216023e-032,  1.10492183e-010,
        5.21126744e-001,  6.98451758e-001,  5.49006469e-008,
        7.45997558e+000,  4.96785934e+000,  5.50594151e+000,
        7.20007614e+000,  1.22370299e-019,  5.74662722e-014,
        1.53848184e-001,  1.70266534e-001,  3.16565244e-008,
        4.52935249e-001,  1.90057960e-001,  2.32190384e-001,
        3.28993927e-002,  1.15986629e-001,  1.33537102e+001,
        6.20996534e+000,  4.11126557e+000,  6.89767697e-001,
        6.57950206e+000,  5.01138620e+000,  5.29482942e+000,
        3.95179893e-016,  3.03867444e-006,  2.75363190e-071,
        1.53713441e-028,  2.42981258e-017,  4.62685175e-104,
        5.24851123e-026,  1.01526317e-003,  9.77096931e-004,
        1.47282066e-003,  2.54490443e-001,  3.37088062e-008,
        5.79671542e-017,  5.90852643e-017,  1.47742493e-003,
        3.16273295e-008,  1.51707962e-006,  5.28002773e-011,
        1.94316774e-001,  4.48372794e-011,  5.75407616e-006,
        6.29313121e-006,  1.07191082e-156,  3.22518280e-069,
        6.45953608e-027,  5.89661082e-004,  3.81338889e-004,
        3.49139298e-001,  2.28307373e-003,  3.37731994e-004,
        5.18744411e-006,  2.29679230e-004,  4.90144024e-015,
        7.89232543e-012,  5.70760500e-016,  5.84862720e-025,
        1.47594322e-158,  1.69555021e-003,  1.21936520e-002,
        1.71217402e-003,  1.14620490e-003,  1.33365812e-003,
        9.00654637e-012,  2.63345382e+000,  1.93223286e-001,
        0.00000000e+000,  3.31022099e+000,  8.49186459e-004,
        6.92560973e-001,  2.32457660e+000,  2.84534534e+000,
        8.85514140e-004,  2.46328676e-001,  4.68466563e+000,
        1.10358723e-003,  9.40980826e-005,  1.38550075e+001,
        9.89772228e-002,  2.40106187e-004,  8.41737525e+000,
        3.10974832e+000,  9.88273420e-026,  1.22729862e-012,
        4.93742780e-020,  1.03814829e-002,  1.46798274e-012,
        8.46374012e-062,  1.06733079e-085,  2.98606309e-037,
        0.00000000e+000,  0.00000000e+000,  1.18588239e-021,
        4.47663279e-105,  2.85767271e-098,  0.00000000e+000,
        1.30341203e-035,  9.26888140e-018,  5.88972980e-005,
        5.16865374e-078,  6.50299074e-028,  7.73577970e-011,
        2.57603703e-001,  0.00000000e+000,  4.10348027e-080,
        2.88924640e-031,  2.45638971e-018,  5.78659855e-063,
        7.58887726e-002,  2.05524312e-013,  0.00000000e+000,
        4.26469187e-003,  2.16092991e-080,  4.53656851e-011,
        7.24940183e-001,  2.53378165e-003,  0.00000000e+000,
        0.00000000e+000,  6.83244198e-080,  1.12669156e-030,
        1.15796258e-028,  8.94566928e-006,  3.36024419e-026,
        7.49154520e-076,  4.76574120e-033,  7.94777692e-003,
        3.75733131e-013,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  1.18312935e-030,  1.50576689e-001,
        4.46962629e-005,  3.19017750e+000,  4.60703029e-028,
        2.67472686e-019,  5.08243692e-031,  4.39222912e-011,
        8.24054618e-018,  2.61836652e-017,  4.79836376e-009,
        5.48898771e-008,  1.27623698e-069,  1.62616009e-103,
        0.00000000e+000,  6.85751162e-028,  0.00000000e+000,
        0.00000000e+000,  2.95994854e-256,  3.21720601e-263,
        4.70350495e-321,  3.23202800e-006,  2.01906192e-004,
        1.24511624e-020,  1.30644998e-052,  1.37554542e-003,
        1.16496389e-030,  8.41756743e-081,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  2.26689970e-004,
        0.00000000e+000,  0.00000000e+000,  2.51722038e-006])), ('train_metrics', OrderedDict([('R2', 0.9954059447356449), ('root_mean_squared_error', 70.84772083357406), ('mean_absolute_error', 52.61313682325343), ('rmse_over_stdev', 0.7443069870284538)])), ('test_metrics', OrderedDict([('R2', 0.014196104575853852), ('root_mean_squared_error', 142.62145741500382), ('mean_absolute_error', 105.60544726215772), ('rmse_over_stdev', 1.4983424449677283)])), ('train_indices', array([   0,    2,    3, ..., 2142, 2143, 2144])), ('test_indices', array([   1,    8,   11,   14,   22,   24,   31,   33,   46,   47,   50,
         57,   65,   67,   72,   75,   76,   82,   88,   93,   96,  101,
        105,  115,  126,  131,  139,  143,  154,  163,  168,  177,  181,
        188,  190,  195,  196,  201,  202,  213,  214,  215,  219,  225,
        226,  243,  246,  248,  260,  271,  275,  283,  287,  292,  298,
        310,  313,  317,  330,  331,  334,  335,  347,  348,  349,  352,
        360,  364,  365,  371,  375,  383,  387,  388,  394,  402,  411,
        412,  419,  420,  421,  424,  425,  430,  439,  440,  444,  451,
        455,  461,  487,  488,  491,  492,  494,  496,  507,  508,  524,
        526,  527,  529,  536,  541,  544,  545,  548,  554,  562,  566,
        568,  572,  573,  580,  588,  590,  593,  596,  598,  602,  603,
        607,  613,  618,  623,  624,  632,  648,  659,  663,  668,  679,
        680,  690,  693,  697,  699,  706,  709,  710,  715,  719,  723,
        725,  726,  729,  758,  761,  764,  767,  772,  778,  787,  814,
        815,  831,  841,  843,  858,  859,  863,  867,  870,  872,  874,
        875,  878,  879,  880,  884,  891,  896,  900,  911,  912,  913,
        916,  920,  924,  925,  936,  941,  942,  944,  951,  978, 1001,
       1007, 1008, 1009, 1028, 1042, 1045, 1050, 1051, 1054, 1062, 1066,
       1067, 1069, 1076, 1081, 1083, 1088, 1089, 1094, 1107, 1115, 1116,
       1124, 1131, 1132, 1135, 1136, 1138, 1139, 1147, 1149, 1150, 1156,
       1163, 1166, 1167, 1174, 1176, 1179, 1180, 1191, 1192, 1194, 1200,
       1204, 1209, 1219, 1229, 1231, 1233, 1239, 1240, 1241, 1251, 1252,
       1258, 1261, 1269, 1271, 1278, 1281, 1284, 1285, 1288, 1291, 1296,
       1304, 1305, 1307, 1308, 1310, 1311, 1316, 1321, 1325, 1331, 1337,
       1345, 1350, 1354, 1356, 1361, 1365, 1376, 1377, 1378, 1380, 1387,
       1388, 1391, 1401, 1414, 1416, 1422, 1439, 1448, 1450, 1451, 1464,
       1472, 1485, 1488, 1492, 1497, 1498, 1499, 1501, 1503, 1505, 1506,
       1507, 1513, 1515, 1521, 1522, 1529, 1535, 1544, 1546, 1551, 1552,
       1553, 1556, 1559, 1561, 1565, 1566, 1573, 1581, 1585, 1589, 1592,
       1611, 1612, 1617, 1620, 1622, 1623, 1628, 1630, 1637, 1655, 1661,
       1662, 1664, 1675, 1676, 1677, 1679, 1682, 1697, 1712, 1714, 1720,
       1722, 1723, 1732, 1735, 1739, 1747, 1760, 1761, 1775, 1791, 1800,
       1804, 1805, 1807, 1815, 1816, 1820, 1830, 1831, 1836, 1847, 1849,
       1859, 1861, 1867, 1873, 1880, 1887, 1896, 1900, 1901, 1904, 1907,
       1908, 1912, 1917, 1927, 1932, 1934, 1937, 1938, 1944, 1952, 1962,
       1963, 1968, 1969, 1970, 1996, 1998, 2007, 2012, 2013, 2017, 2025,
       2027, 2031, 2045, 2047, 2049, 2050, 2052, 2057, 2059, 2066, 2070,
       2073, 2074, 2079, 2087, 2088, 2091, 2092, 2093, 2094, 2097, 2099,
       2102, 2104, 2108, 2111, 2118, 2122, 2123, 2127, 2129, 2137, 2141])), ('train_groups', None), ('test_groups', None), ('prediction_metrics', None)])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge/RepeatedKFold/best_worst_split.png'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_best_worst_split(y_true, best_run, worst_run, savepath, label)
display(Image(filename='best_worst_split.png'))
